In [1]:
import dask

import pandas as pd
import numpy as np
import dask.dataframe as dd

from dask import compute
from dask.distributed import Client

nan = np.nan
pd.options.display.max_columns = None
client = dask.distributed.Client()


In [2]:
path = 's3://hackathon-bbva-parceros/t_mdco_tfra400_Base3M_1.0.csv'

In [3]:
%%time
df = dd.read_csv(path,
                 dtype={"producto_1_number":str,
                        "bin_2_number":str, 
                        "NSS":str,
                        "pasaporte":str,
                        "clave_id":str,
                        "clave_secundaria_text": str
                       })

CPU times: user 283 ms, sys: 44.2 ms, total: 327 ms
Wall time: 1.55 s


In [4]:
df.head()

,id_cliente,NU_CTE,CANAL_ENTRADA,clave_id,desc_text,producto_id,correo_id,CVV_secreto,CURP,CODIGO_BANCO,REFERENCIA_PAGO,clave_secundaria_text,desc_id,clave_primaria_text,producto_1_number,tel_id,bin_2_number,SUBCANAL_ENTRANTE,CODIGO_ENTIDAD_ORI,nombre_text,apellid@_text,INSTRUMENTO_PAGO,pasaporte,TERMINAL,FECHA_ULTIMO_USO,NSS,REFERENCIA_NUMERICA,MOTIVO_PAGO,INFE_id,registro.xls,TIPO_TARJETA,agrupacion_id
0,1,B3477219,BNET,640402,iolandesnaddon,AD3,tnioPmirG91,495,KLX84,RQBI,TBHGUDCEQLAVKRONSXYP,+66 7ZA 76B 7BS6,IoSn,SE(SBS)A7A-61ZS,1000010110,19-7517 5194,0100000101,4,74,isnaddon0,prlog.org,paperbacks,1577F7EB349596F,KNtramps,1972-07-10,42220146665,3685710,S80,jmx0641086212,Version196404.xls,TDC,HDF
1,2,W0818632,BMOVIL,820713,jemiepeachman,MS9,enaLedisyaB18,759,LDH08,LOAE,IHXKSUZENGVPRYDQFOJA,+ESl EZA 6l7 AAE6,JePe,ZE0(76E)96A-07A6,1000010110,97-9572 4516,0101010100,4,74,jpeachman1,people!.cn,comestible,NaN,KNunseen,1998-08-22,1026036203075,5754645,Thunderbird,uix4568194236,Version198207.xls,TDC,MDF
2,3,Z4396398,BNET,580905,justusmurton,NU7,kraPrelliM2518,739,UMM71,QOFZ,QGCDLKSXNBIVAPZOJREY,+6Z ZSl 7ZA AEEO,JuMu,B6(S9E)BAE-0B6A,1000011101,17-7600 8904,1010000101,4,74,jmurton2,pen.io,windmilled,7D3C2D442B0D6A,KNcatnap,1991-10-28,645036454164,222503,GTO,vwu2996077629,Version195809.xls,TDC,HDF
3,4,L1839991,BNET,530805,foresternanni,EU6,teertSshcaS171,244,RRL32,TQKJ,IZFUHGOTKXWBLVYDQMPS,+SS A69 Z9Z 7SB7,FoNa,6E(SZ9)B6S-ZEZ0,1111000011,87-6325 2840,001101001,4,74,fnanni3,ocn.ne.jp,achievable,NaN,KNbastes,1979-09-03,350710400652,4545783,Spyder,jju5328474945,Version195308.xls,TDC,MDF
4,5,Z4139418,BNET,800731,nathaliachiles,UC7,lliHtserclezaH854,970,SYX28,XBFR,OTCJVQAMPFXKBSHGIRYU,+96A 9BO B97 lOll,NaCh,B6(61Z)B7Z-ESEE,1000010011,98-3249 7495,0100110101,4,74,nchiles4,dell!,periodical,173E41CD56A7D2,KNatom's,1996-09-13,347443313710,1294003,LX,rzc9286783184,Version198007.xls,TDC,HDF


In [5]:
%%time
# Direccion
df['direccion'] = df.loc[:,'correo_id'].apply(lambda x: x[::-1],meta=('correo_id', 'object'))

CPU times: user 10.1 ms, sys: 3.87 ms, total: 14 ms
Wall time: 13.9 ms


In [6]:
%%time
# Obtener nombre completo
def build_name(row):
    name = row['desc_text']
    lastShort = row['desc_id'][2:]
    index = name.find(lastShort.lower())
    return f'{name[:index].capitalize()} {name[index:].capitalize()}'
df['full_name'] = df[['desc_text', 'desc_id']].apply(build_name, axis=1, meta=('correo_id', 'object'))

CPU times: user 12.9 ms, sys: 0 ns, total: 12.9 ms
Wall time: 12.9 ms


In [7]:
%%time
# Obtener email completo
def build_email(row):
    user = row['nombre_text']
    domain = str(row['apellid@_text']).replace('!','.com')
    return f'{user}@{domain}'
df['email'] = df[['nombre_text', 'apellid@_text']].apply(build_email, axis=1, meta=('correo_id', 'object'))

CPU times: user 9.81 ms, sys: 3.6 ms, total: 13.4 ms
Wall time: 14 ms


In [8]:
%%time
# Obtener los 17 numeros del pasaporte
def build_passport(value):
    if str(value) == 'nan' or value == nan:
        return ''
    else:
        return str(int(str(value), 16)).zfill(17)
df['pasaporte_'] = df['pasaporte'].apply(
    build_passport,
    meta=('pasaporte_', 'object')
)

CPU times: user 11.2 ms, sys: 368 µs, total: 11.6 ms
Wall time: 11.2 ms


In [9]:
%%time
# Obtener los 17 numeros del pasaporte
def build_nss(value):
    if str(value) == 'nan' or value == nan:
        return ''
    else:
        return str(int(str(value), 8)).zfill(17)
df['NSS_'] = df['NSS'].apply(
    build_nss,
    meta=('NSS_', 'object')
)

CPU times: user 11.3 ms, sys: 0 ns, total: 11.3 ms
Wall time: 10.9 ms


In [10]:
# Obtener numeros de telefono
mapper = {
    'A' : '4',
    'B' : '8',
    'Z' : '2',
    'I' : '1',
    'S' : '5',
    'E' : '3',
    'l' : '1',
    'O' : '0'
}
def unleet(string):
    res = ''
    if str(string) == 'nan' or string == nan:
        return ''
    else:
        for c in string: res += mapper.get(c, c)
        return res
df['phone1'] = df['clave_secundaria_text'].apply(unleet, meta=('telefono_1', 'object'))
df['phone2'] = df['clave_primaria_text'].apply(unleet, meta=('telefono_2', 'object'))


CPU times: user 22.7 ms, sys: 0 ns, total: 22.7 ms
Wall time: 21.7 ms


In [11]:
%%time
# Obtener fecha de algo
def get_date(row):
    clave = row['clave_id'][-2:]
    registro = row['registro.xls'][7:-4]
    return f'{registro[:4]}-{registro[4:]}-{clave}'
df['date0'] = df[['clave_id', 'registro.xls']].apply(get_date, axis=1, meta=('date0', 'object'))

CPU times: user 15.1 ms, sys: 0 ns, total: 15.1 ms
Wall time: 14.1 ms


In [12]:
def validate(value):
    return str(value) != 'nan' and value != nan

In [13]:
# Tarjeta
def build_tarjeta(row):
    a = row['producto_1_number']
    b = row['bin_2_number']
    c = row['INFE_id']
    for x in (a,b,c):
        if not validate(x):
            return ''
    
    return f'{int(a, 2)}{str(int(b, 2)).zfill(3)}{str(c[3:]).zfill(10)}'
    
df['tarjeta'] = df[['producto_1_number', 'bin_2_number','INFE_id']].apply(
    build_tarjeta,
    axis=1, 
    meta=('tarjeta', 'object')
)

In [14]:
# CURP
def get_curp(row):
    pass

In [15]:
%%time
res_df = df[['direccion', 'full_name', 'email', 'pasaporte_', 'NSS_', 'phone1', 'phone2', 'date0','tarjeta']]

CPU times: user 1.86 ms, sys: 0 ns, total: 1.86 ms
Wall time: 1.83 ms


In [16]:
res_df =res_df.compute()

In [18]:
res_df

,direccion,full_name,email,pasaporte_,NSS_,phone1,phone2,date0,tarjeta
0,19GrimPoint,Iolande Snaddon,isnaddon0@prlog.org,96686099162290543,00000004601204149,+66 724 768 7856,53(585)474-6125,1964-04-02,5342610641086212
1,81BaysideLane,Jemie Peachman,jpeachman1@people.com.cn,,00000071680198205,+351 324 617 4436,230(763)964-0746,1982-07-13,5343404568194236
2,8152MillerPark,Justus Murton,jmurton2@pen.io,35250537203699050,00000056513681524,+62 251 724 4330,86(593)843-0864,1958-09-05,5416452996077629
3,171SachsStreet,Forester Nanni,fnanni3@ocn.ne.jp,,00000031258182058,+55 469 292 7587,63(529)865-2320,1953-08-05,9631055328474945
4,458HazelcrestHill,Nathalia Chiles,nchiles4@dell.com,06542376803149778,00000031080683464,+964 980 897 1011,86(612)872-3533,1980-07-31,5313099286783184
...,...,...,...,...,...,...,...,...,...
150781,5BrentwoodDrive,Zara Dresser,zdresser255n@bigcartel.com,72916483490239047,00000002013657870,+230 536 393 7818,86(339)948-7180,1943-05-31,9320302452944829
150782,510VictoriaStreet,Skipper Mander,smander255o@geocities.jp,20435652049221291,00000079899154557,+20 610 110 4446,351(564)236-3879,1983-12-11,9321083038445815
150783,7GracelandJunction,Debora Mounce,dmounce255p@time.com,,00000076274686747,+54 375 388 6061,93(575)255-3836,1988-09-03,8893777225031133
150784,618ThiererCenter,Alanah Holde,aholde255q@bandcamp.com,43855354376114202,00000026357308824,+351 954 450 7594,55(694)848-4220,1995-01-01,8893770962206429
